## **Códigos en Python**

Escribe en Python un codigo que obtenga la factorización *LU* otro que obtenga la factorizacion *PLU* y otro que obtenga la
factorizacion de Cholesky de una matriz *A*. En caso de que la factorizacion no pueda realizarse sobre la matriz *A* tu codigo 
debe arrojar que es lo que la matriz *A* no cumple.

In [62]:
import numpy as np
import matplotlib.pyplot as plt

In [63]:
class InvalidInputMatrix(Exception):
    def __init__(self, message):
        super().__init__(message)

In [64]:
def lu_fact(A: np.ndarray) -> tuple[np.ndarray, np.ndarray]:
    """
    LU decomposition of a square matrix A. 
    Matrix A must not contain any zero in the diagonal and must be invertible.

    Example
    --------
    >>> A = np.array([[3, 1, 1], [-3, -3, 1], [3, -3, 6]])
    >>> lu_fact(A)
    (array([[ 1.,  0.,  0.],
            [-1.,  1.,  0.],
            [ 1.,  2.,  1.]]), 
    array([[ 3,  1,  1],
          [ 0, -2,  2],
          [ 0,  0,  1]]))

    Parameters
    ----------
    A (ndarray) : nxn coefficient matrix

    Returns
    -------
    U (ndarray) : Upper triangular matrix
    L (ndarray) : Lower triangular matrix
    """
    if np.linalg.det(A) == 0:
        raise InvalidInputMatrix('The matrix must be invertible.')
    m, n = np.shape(A)
    if m != n:
        raise InvalidInputMatrix('The matrix must be square.')
    if np.any(np.diag(A) == 0):
        raise InvalidInputMatrix('The matrix must not contain any zero in the diagonal.')
    L = np.zeros((n, n))
    U = np.copy(A)
    for j in range(n-1):
        for i in range(j+1,n):
            L[i,j] = (U[i,j])/(U[j,j])
            U[i] = U[i] - L[i,j]*U[j]
    L = L + np.eye(n)
    return L, U

In [65]:
def swap_rows(A: np.ndarray, i: int, j: int) -> None:
    """ 
    Swap rows i and j in matrix A.
    """
    temp = np.copy(A[i])
    A[i] = A[j]
    A[j] = temp

In [66]:
def plu_fact(A: np.ndarray) -> tuple[np.ndarray, np.ndarray, np.ndarray]:
    """ 
    PLU decomposition of a square matrix A

    Example
    --------
    >>> A = np.array([[0., -1., 4.], [2., 1., 1.], [1., 1., -2.]])
    >>> plu_fact(A)
    (array([[ 0. , 1. , 0.],
            [ 1. , 0. , 0.],
            [ 0. , 0. , 1.]]),
    array([[ 1. ,  0. ,  0. ],
           [ 0. ,  1. ,  0. ],
           [ 0.5, -0.5,  1. ]]),
    array([[ 2. ,  1. ,  1. ],
           [ 0. , -1. ,  4. ],
           [ 0. ,  0. , -0.5]]))

    Parameters
    ----------
    A (ndarray) : nxn coefficient matrix

    Returns
    -------
    P (ndarray) : Permutation matrix
    U (ndarray) : Upper triangular matrix
    L (ndarray) : Lower triangular matrix
    """
    if np.linalg.det(A) == 0:
        raise InvalidInputMatrix('The matrix must be invertible.')
    m, n = np.shape(A)
    if m != n:
        raise InvalidInputMatrix('The matrix must be square.')
    P = np.eye(n)
    L = np.zeros((n, n))
    U = np.copy(A)
    for j in range(n-1):
        if U[j,j] == 0:
            max_num = 0
            index = 0
            for k in range(j+1, n):
                if abs(U[k,j]) > max_num:
                    max_num = abs(U[k,j])
                    index = k
            swap_rows(U, j, index)
            swap_rows(P, j, index)
        for i in range(j+1,n):
            L[i,j] = (U[i,j])/(U[j,j])
            U[i] = U[i] - L[i,j]*U[j]
    L = L + np.eye(n)
    return P, L, U

In [90]:
def cholesky_fact(A: np.ndarray) -> np.ndarray:
    """ 
    Cholesky decomposition of a symmetric positive definite matrix A

    Example
    --------
    >>> A = np.array([[4., -1., 1.], [-1., 4.25, 2.75], [1., 2.75, 3.5]])
    >>> cholesky_fact(A)
    array([[ 2. , -0.5,  0.5],
           [ 0. ,  2. ,  1.5],
           [ 0. ,  0. ,  1. ]])

    Parameters
    ----------
    A (ndarray) : nxn coefficient matrix

    Returns
    -------
    U (ndarray) : Upper triangular matrix
    """
    if np.linalg.det(A) == 0:
        raise InvalidInputMatrix('The matrix must be invertible.')
    m, n = np.shape(A)
    if m != n:
        raise InvalidInputMatrix('The matrix must be square.')
    if not np.allclose(A, A.T):
        raise InvalidInputMatrix('The matrix must be symmetric.')
    x = np.ones(n)
    if x.T @ A @ x <= 0:
        raise InvalidInputMatrix('The matrix must be positive definite.')
    U = np.copy(A)
    for i in range(n):
        for j in range(i+1, n):
            U[j] = U[j] - U[i]*U[j,i]/U[i,i]
        U[i] = U[i]/np.sqrt(U[i,i])
    return U

## **Error en la factorización**

Se define el error en la factorización LU como

$$ ||A-LU|| $$

para la factorización PLU como

$$ ||PA-LU|| $$

y para la factorización de Cholesky como

$$ || A-U^TU || $$

In [ ]:
def lu_error(A: np.ndarray, L: np.ndarray, U: np.ndarray) -> float:
    """ 
    Calculate the error of LU decomposition

    Parameters
    ----------
    A (ndarray) : nxn coefficient matrix
    L (ndarray) : Lower triangular matrix
    U (ndarray) : Upper triangular matrix

    Returns
    -------
    error (float) : Error of LU decomposition
    """
    return np.linalg.norm(A - L @ U)

In [ ]:
def plu_error(A: np.ndarray, P: np.ndarray, L: np.ndarray, U: np.ndarray) -> float:
    """ 
    Calculate the error of PLU decomposition

    Parameters
    ----------
    A (ndarray) : nxn coefficient matrix
    P (ndarray) : Permutation matrix
    L (ndarray) : Lower triangular matrix
    U (ndarray) : Upper triangular matrix

    Returns
    -------
    error (float) : Error of PLU decomposition
    """
    return np.linalg.norm(P @ A - L @ U)

In [ ]:
def cholesky_error(A: np.ndarray, U: np.ndarray) -> float:
    """ 
    Calculate the error of Cholesky decomposition

    Parameters
    ----------
    A (ndarray) : nxn coefficient matrix
    U (ndarray) : Upper triangular matrix

    Returns
    -------
    error (float) : Error of Cholesky decomposition
    """
    return np.linalg.norm(A - U.T @ U)

1. Considera un vector N con 50 entradas en orden ascendente, de tal forma que si $N=(n_1,...,n_{50})$, entonces $n_1=2, n_i-n_{i-1}=10$ y en consecuencia $n_{50} = 502$.

In [96]:
N = np.arange(2, 503, 10)
N

array([  2,  12,  22,  32,  42,  52,  62,  72,  82,  92, 102, 112, 122,
       132, 142, 152, 162, 172, 182, 192, 202, 212, 222, 232, 242, 252,
       262, 272, 282, 292, 302, 312, 322, 332, 342, 352, 362, 372, 382,
       392, 402, 412, 422, 432, 442, 452, 462, 472, 482, 492, 502])

2. Para cada $n_i$, crea una matriz $M_i$ de dimensiones $n \times n$, con valores aleatorios y calcula el error asociado a la factorización

## **Estabilidad en la factorización PLU**